# Athena codes discovery

- Jingna Feng
- 4/15/2025 


1. scispacy core large model:identify entities (python env scispacy, python 3.8) -- infodev
2. map entities with SNOMED thru pymedtermino package (python env pt36, python 3.6)
3. map entities with Athena thru pandas package (python env base, python 3.11.7)
4. Concat both results to a csv of named entities identified (python env base, python 3.11.7)

In [1]:
import pandas as pd

import numpy as np

pd.set_option('display.max_colwidth', None)


In [2]:
!ls -lh /Users/feng.jingna/macLocal/Vocabulary/vocabOntoCodex

total 2616600
-rw-rw-r--@ 1 feng.jingna  staff   253M Apr 15 14:59 CONCEPT.csv
-rw-rw-r--@ 1 feng.jingna  staff   228M Apr 15 15:01 CONCEPT_ANCESTOR.csv
-rw-rw-r--@ 1 feng.jingna  staff    18K Apr 15 14:59 CONCEPT_CLASS.csv
-rw-rw-r--@ 1 feng.jingna  staff   618M Apr 15 15:03 CONCEPT_RELATIONSHIP.csv
-rw-rw-r--@ 1 feng.jingna  staff   168M Apr 15 15:00 CONCEPT_SYNONYM.csv
-rw-rw-r--@ 1 feng.jingna  staff   1.7K Apr 15 14:59 DOMAIN.csv
-rw-rw-r--@ 1 feng.jingna  staff   9.9M Apr 15 15:00 DRUG_STRENGTH.csv
-rw-rw-r--@ 1 feng.jingna  staff    52K Apr 15 14:59 RELATIONSHIP.csv
-rw-rw-r--@ 1 feng.jingna  staff   3.6K Apr 15 15:00 VOCABULARY.csv


## Vocabulary dictionary

In [3]:
path_out = '/Users/feng.jingna/macLocal/Athena/output/'
in_path = '/Users/feng.jingna/macLocal/Vocabulary/vocabOntoCodex/'

In [4]:
voc = pd.read_csv(in_path+'CONCEPT.csv',low_memory=False,delimiter='\t')
voc.head()

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,3541502,Adverse reaction to drug primarily affecting the autonomic nervous system NOS,Condition,SNOMED,Disorder,NaN,694331000000106,20100401,20111003,D
1,3542356,Adverse reaction to other central nervous system depressants and anaesthetics,Condition,SNOMED,Disorder,NaN,705311000000105,20100401,20111003,D
2,42538812,Somatic hallucination,Condition,SNOMED,Clinical Finding,S,762620006,20180131,20991231,NaN
3,40629514,Stillbirth,Condition,SNOMED,Clinical Finding,NaN,76358005,20020131,20020131,D
4,3539866,"Adverse reaction to other skin, mucous membrane, ophthalmological, otorhinolaryngological and dental drugs",Condition,SNOMED,Disorder,NaN,671311000000108,20100401,20111003,D


In [5]:
voc.vocabulary_id.unique()

array(['SNOMED', 'ATC', 'CDM', 'Concept Class', 'Condition Status',
       'Condition Type', 'Cost', 'Cost Type', 'Death Type', 'Device Type',
       'Domain', 'Drug Type', 'ICD10CM', 'Vocabulary', 'Visit', 'UCUM',
       'Relationship', 'Episode', 'RxNorm', 'Korean Revenue Code',
       'LOINC', 'Language', 'Meas Type', 'Metadata', nan, 'Note Type',
       'OSM', 'Obs Period Type', 'Observation Type', 'Plan',
       'Plan Stop Reason', 'Procedure Type', 'SOPT', 'Sponsor',
       'Type Concept', 'UB04 Point of Origin', 'UB04 Pri Typ of Adm',
       'UB04 Pt dis status', 'UB04 Typ bill', 'US Census', 'Visit Type'],
      dtype=object)

In [6]:
## Keep valid codes only

nl = ['D', 'U']

voc = voc.loc[~voc['invalid_reason'].isin(nl)].reset_index(drop=True)
voc = voc.loc[~voc['concept_name'].astype(str).str.contains('retired',case=False)].reset_index(drop=True)

voc.shape

(1166829, 10)

In [15]:
voc.domain_id.value_counts()

domain_id
Drug                   257701
Observation            215027
Geography              204012
Condition              192964
Measurement            145060
Procedure               58606
Spec Anatomic Site      37078
Meas Value              24376
Device                  15086
Note                     4118
Metadata                 3849
Unit                     2279
Type Concept             2150
Specimen                 1897
Language                  840
Provider                  548
Visit                     377
Relationship              210
Race                      185
Payer                     167
Route                     163
Cost                       51
Condition Status           22
Episode                    14
Plan Stop Reason           13
Plan                       11
Revenue Code                7
Meas Value Operator         7
Sponsor                     6
Gender                      5
Name: count, dtype: int64

In [7]:
voc.head()

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,42538812,Somatic hallucination,Condition,SNOMED,Clinical Finding,S,762620006,20180131,20991231,NaN
1,4084170,Non-allergic anaphylaxis caused by whole blood,Condition,SNOMED,Disorder,S,241944009,20020131,20991231,NaN
2,4085530,Unformed visual hallucinations,Condition,SNOMED,Clinical Finding,S,247733004,20020131,20991231,NaN
3,4085038,Formed visual hallucinations,Condition,SNOMED,Clinical Finding,S,247734005,20020131,20991231,NaN
4,4085531,Scenic visual hallucinations,Condition,SNOMED,Clinical Finding,S,247735006,20020131,20991231,NaN


In [19]:
voc.loc[voc['concept_code']=='76282-3']

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
798457,21490872,Heart rate.beat-to-beat by EKG,Measurement,LOINC,Clinical Observation,S,76282-3,20160624,20991231,NaN


In [20]:
voc.loc[voc['concept_code']=='33878-0']

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
99062,3042833,Mean blood flow rate by US,Measurement,LOINC,Clinical Observation,S,33878-0,19700101,20991231,NaN


In [22]:
voc.loc[voc['concept_name'].str.contains('2D',na=False)]

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
21850,36308116,CYP2D6*1A,Meas Value,LOINC,Answer,S,LA19861-6,19700101,20991231,NaN
21851,36308244,CYP2D6*1B,Meas Value,LOINC,Answer,S,LA19862-4,19700101,20991231,NaN
21852,36310412,CYP2D6*1C,Meas Value,LOINC,Answer,S,LA19863-2,19700101,20991231,NaN
21853,36309761,CYP2D6*1E,Meas Value,LOINC,Answer,S,LA19865-7,19700101,20991231,NaN
21854,36310475,CYP2D6*1XN,Meas Value,LOINC,Answer,S,LA19866-5,19700101,20991231,NaN
...,...,...,...,...,...,...,...,...,...,...
1165463,21495343,Aortic valve Vena contracta diameter during diastole by US.2D+doppler,Measurement,LOINC,Clinical Observation,S,79948-6,20160624,20991231,NaN
1165464,21494028,Left ventricular Ejection fraction by US.2D+Calculated by biplane method of disks,Measurement,LOINC,Clinical Observation,S,79991-6,20160624,20991231,NaN
1165863,3022786,Right ventricular Cardiac index by 2D echo,Measurement,LOINC,Clinical Observation,S,8755-1,19700101,20991231,NaN
1165899,3008533,Left ventricular Stroke volume by 2D echo,Measurement,LOINC,Clinical Observation,S,8765-0,19700101,20991231,NaN


In [16]:
## Test finding EKG measurement

ekg = voc.loc[voc['domain_id']=='Measurement'].reset_index(drop=True)

ekg.vocabulary_id.value_counts()

vocabulary_id
LOINC      113667
SNOMED      31191
ICD10CM       202
Name: count, dtype: int64

In [24]:
ekg.loc[ekg['concept_name'].str.contains('2D|3D',na=False)]

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
3702,46286617,"HoNOS-LD (Health of the Nation Outcome Scales for People with Learning Disabilities) rating scale 3D score - anxiety, phobias, obsessive or compulsive behaviour",Measurement,SNOMED,Observable Entity,NaN,988291000000107,20151001,20991231,NaN
17100,3037208,Left atrial Area by US.2D.A4C,Measurement,LOINC,Clinical Observation,S,17977-0,19700101,20991231,NaN
17419,3012724,Left ventricular Maximum Apical 2 chamber.diastole [Volume] by US 2D modified,Measurement,LOINC,Clinical Observation,S,17986-1,19700101,20991231,NaN
17421,3015670,Left ventricular Apical 2 chamber.systole.minimum [Volume] by US 2D modified,Measurement,LOINC,Clinical Observation,S,17987-9,19700101,20991231,NaN
17424,3008412,Right atrial Apical 4 chamber [Area] by US 2D,Measurement,LOINC,Clinical Observation,S,17988-7,19700101,20991231,NaN
...,...,...,...,...,...,...,...,...,...,...
144491,21495343,Aortic valve Vena contracta diameter during diastole by US.2D+doppler,Measurement,LOINC,Clinical Observation,S,79948-6,20160624,20991231,NaN
144492,21494028,Left ventricular Ejection fraction by US.2D+Calculated by biplane method of disks,Measurement,LOINC,Clinical Observation,S,79991-6,20160624,20991231,NaN
144727,3022786,Right ventricular Cardiac index by 2D echo,Measurement,LOINC,Clinical Observation,S,8755-1,19700101,20991231,NaN
144763,3008533,Left ventricular Stroke volume by 2D echo,Measurement,LOINC,Clinical Observation,S,8765-0,19700101,20991231,NaN


In [25]:
ekg.shape

(145060, 10)

In [18]:
ekg.head()

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,1102830,Progesterone receptor status,Measurement,ICD10CM,4-char nonbill code,NaN,Z17.2,20241001,20991231,NaN
1,1102833,Human epidermal growth factor 2 receptor,Measurement,ICD10CM,4-char nonbill code,NaN,Z17.3,20241001,20991231,NaN
2,1102834,Human epidermal growth factor receptor 2 positive status,Measurement,ICD10CM,5-char billing code,NaN,Z17.31,20241001,20991231,NaN
3,1102835,Human epidermal growth factor receptor 2 negative status,Measurement,ICD10CM,5-char billing code,NaN,Z17.32,20241001,20991231,NaN
4,1102836,Combined receptor status,Measurement,ICD10CM,4-char nonbill code,NaN,Z17.4,20241001,20991231,NaN


In [26]:
ekg.domain_id.value_counts()

domain_id
Measurement    145060
Name: count, dtype: int64

In [27]:
ekg.columns

Index(['concept_id', 'concept_name', 'domain_id', 'vocabulary_id',
       'concept_class_id', 'standard_concept', 'concept_code',
       'valid_start_date', 'valid_end_date', 'invalid_reason'],
      dtype='object')

In [28]:
ekg = ekg[['concept_id', 'concept_name', 'domain_id', 'vocabulary_id',
       'concept_class_id', 'standard_concept', 'concept_code']].reset_index(drop=True)
ekg.info

<bound method DataFrame.info of         concept_id                                              concept_name  \
0          1102830                              Progesterone receptor status   
1          1102833                  Human epidermal growth factor 2 receptor   
2          1102834  Human epidermal growth factor receptor 2 positive status   
3          1102835  Human epidermal growth factor receptor 2 negative status   
4          1102836                                  Combined receptor status   
...            ...                                                       ...   
145055     1618050                   Vespa velutina Ab.IgE | Serum | Allergy   
145056     1991155                      Class | Pacemaker lead | General EKG   
145057    37047169                                              HLA antigens   
145058     1259466   Mycobacterium leprae Ab. anti-PGL-1 Ab [Titer] in Serum   
145059     1261334                          Hemoglobin|Moment in time|BldCoV   

       

In [29]:
ekg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145060 entries, 0 to 145059
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   concept_id        145060 non-null  int64 
 1   concept_name      145060 non-null  object
 2   domain_id         145060 non-null  object
 3   vocabulary_id     145060 non-null  object
 4   concept_class_id  145060 non-null  object
 5   standard_concept  139191 non-null  object
 6   concept_code      145060 non-null  object
dtypes: int64(1), object(6)
memory usage: 7.7+ MB


In [31]:
ekg['concept_id'] = ekg['concept_id'].astype(str)
ekg['standard_concept'] = ekg['standard_concept'].fillna('UNK')
ekg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145060 entries, 0 to 145059
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   concept_id        145060 non-null  object
 1   concept_name      145060 non-null  object
 2   domain_id         145060 non-null  object
 3   vocabulary_id     145060 non-null  object
 4   concept_class_id  145060 non-null  object
 5   standard_concept  145060 non-null  object
 6   concept_code      145060 non-null  object
dtypes: object(7)
memory usage: 7.7+ MB


In [10]:
# Make SNOMED CT table

snm = voc.loc[voc['vocabulary_id']=='SNOMED'].reset_index(drop=True)

snm.shape

(409111, 10)

In [11]:
snm

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,42538812,Somatic hallucination,Condition,SNOMED,Clinical Finding,S,762620006,20180131,20991231,NaN
1,4084170,Non-allergic anaphylaxis caused by whole blood,Condition,SNOMED,Disorder,S,241944009,20020131,20991231,NaN
2,4085530,Unformed visual hallucinations,Condition,SNOMED,Clinical Finding,S,247733004,20020131,20991231,NaN
3,4085038,Formed visual hallucinations,Condition,SNOMED,Clinical Finding,S,247734005,20020131,20991231,NaN
4,4085531,Scenic visual hallucinations,Condition,SNOMED,Clinical Finding,S,247735006,20020131,20991231,NaN
...,...,...,...,...,...,...,...,...,...,...
409106,1247089,Opiate in urine by confirmatory technique,Measurement,SNOMED,Observable Entity,S,376261000119107,20230901,20991231,NaN
409107,1247090,Assessment using Asthma Therapy Assessment Questionnaire for children and adolescents,Measurement,SNOMED,Procedure,NaN,751071000124103,20240301,20991231,NaN
409108,1247091,Assessment using Asthma Therapy Assessment Questionnaire Adult,Measurement,SNOMED,Procedure,NaN,751081000124100,20240301,20991231,NaN
409109,1247092,Asthma Therapy Assessment Questionnaire Adult score,Measurement,SNOMED,Observable Entity,NaN,751091000124102,20240301,20991231,NaN


In [12]:
snm.shape

(409111, 10)

In [13]:
snm.concept_class_id.value_counts()

concept_class_id
Disorder               91495
Procedure              70130
Clinical Finding       43627
Body Structure         37078
Organism               34335
Substance              27770
Observable Entity      20856
Physical Object        13969
Qualifier Value        11792
Pharma/Biol Product    10064
Clinical Drug           8349
Context-dependent       7260
Clinical Drug Form      7251
Morph Abnormality       5136
Social Context          4880
Event                   3525
Staging / Scales        2284
Location                1960
Specimen                1897
Record Artifact         1741
Attribute               1199
Navi Concept             725
Dose Form                513
Model Comp               388
Disposition              379
Namespace Concept        325
Physical Force           172
Linkage Assertion          9
Special Concept            1
Linkage Concept            1
Name: count, dtype: int64

In [14]:
snm.domain_id.value_counts()

domain_id
Observation            117740
Condition              105601
Procedure               48709
Drug                    40702
Spec Anatomic Site      37078
Measurement             31191
Device                  15086
Meas Value               4084
Specimen                 1897
Type Concept             1741
Metadata                 1439
Unit                     1240
Language                  839
Geography                 660
Provider                  548
Relationship              196
Race                      185
Route                     163
Meas Value Operator         7
Gender                      5
Name: count, dtype: int64

In [11]:
snm.concept_class_id.unique()

array(['Clinical Finding', 'Disorder', 'Substance', 'Observable Entity',
       'Clinical Drug', 'Procedure', 'Context-dependent', 'Organism',
       'Dose Form', 'Qualifier Value', 'Clinical Drug Form',
       'Body Structure', 'Morph Abnormality', 'Pharma/Biol Product',
       'Staging / Scales', 'Disposition', 'Record Artifact', 'Specimen',
       'Namespace Concept', 'Event', 'Social Context', 'Physical Object',
       'Model Comp', 'Location', 'Physical Force', 'Attribute',
       'Navi Concept', 'Special Concept', 'Linkage Concept',
       'Linkage Assertion'], dtype=object)

In [12]:
## Use case: Identify Insulin Lispro's SNOMED

snm.loc[snm['concept_name'].astype(str).str.contains('Insulin lispro', case=False)]

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
22863,37157824,Insulin lispro 200 unit/mL solution for injection,Drug,SNOMED,Clinical Drug,NaN,1236783005,20220731,20991231,NaN
40075,604560,Insulin lispro and insulin lispro protamine only product in parenteral dose form,Drug,SNOMED,Clinical Drug Form,NaN,1002247002,20210131,20991231,NaN
170812,4304623,Insulin lispro-containing product,Drug,SNOMED,Pharma/Biol Product,NaN,388454007,20030131,20991231,NaN
182967,4259079,Insulin lispro,Drug,SNOMED,Substance,NaN,412210000,20040731,20991231,NaN
182968,4135917,Insulin lispro protamine- and insulin lispro-containing product,Drug,SNOMED,Pharma/Biol Product,NaN,412211001,20040731,20991231,NaN
193273,4144466,Insulin lispro protamine,Drug,SNOMED,Substance,NaN,426313003,20070731,20991231,NaN
269408,35625808,Insulin lispro-containing product in parenteral dose form,Drug,SNOMED,Clinical Drug Form,NaN,768894000,20180731,20991231,NaN
274554,36678931,Insulin lispro protamine only product,Drug,SNOMED,Pharma/Biol Product,NaN,776345003,20190131,20991231,NaN
274759,36679137,Insulin lispro only product,Drug,SNOMED,Pharma/Biol Product,NaN,776551009,20190131,20991231,NaN
277927,36682152,Insulin lispro only product in parenteral dose form,Drug,SNOMED,Clinical Drug Form,NaN,779721001,20190131,20991231,NaN


In [18]:
# Make LOINC table

loi = voc.loc[voc['vocabulary_id']=='LOINC'].reset_index(drop=True)

loi.shape

(235217, 10)

In [19]:
loi.head()

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,3011313,Oxygen content in Right atrium,Measurement,LOINC,Lab Test,S,10234-3,19700101,20991231,NaN
1,3002526,Oxygen content in Pulmonary wedge,Measurement,LOINC,Lab Test,S,10247-5,19700101,20991231,NaN
2,3011287,Niacin [Mass/volume] in Blood,Measurement,LOINC,Lab Test,S,10836-5,19970303,20991231,NaN
3,3020785,Consult note,Note,LOINC,Clinical Observation,S,11488-4,19700101,20991231,NaN
4,3019248,Fetal Tricuspid valve E-wave/A-wave US.doppler,Measurement,LOINC,Clinical Observation,S,11651-7,19700101,20991231,NaN


In [20]:
## Check if concept ID is an index to match different vocabularies

loi_snm = loi.merge(snm, on='concept_id', how='inner')
loi_snm

,concept_id,concept_name_x,domain_id_x,vocabulary_id_x,concept_class_id_x,standard_concept_x,concept_code_x,valid_start_date_x,valid_end_date_x,invalid_reason_x,concept_name_y,domain_id_y,vocabulary_id_y,concept_class_id_y,standard_concept_y,concept_code_y,valid_start_date_y,valid_end_date_y,invalid_reason_y


In [21]:
# Make ATC CT table

atc = voc.loc[voc['vocabulary_id']=='ATC'].reset_index(drop=True)

atc.shape

(6897, 10)

In [22]:
atc.head()

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,21600001,ALIMENTARY TRACT AND METABOLISM,Drug,ATC,ATC 1st,C,A,19700101,20991231,NaN
1,21600002,STOMATOLOGICAL PREPARATIONS,Drug,ATC,ATC 2nd,C,A01,19700101,20991231,NaN
2,21600003,STOMATOLOGICAL PREPARATIONS,Drug,ATC,ATC 3rd,C,A01A,19700101,20991231,NaN
3,21600004,Caries prophylactic agents,Drug,ATC,ATC 4th,C,A01AA,19700101,20991231,NaN
4,21600008,"stannous fluoride; oral, local oral",Drug,ATC,ATC 5th,C,A01AA04,19700101,20991231,NaN


In [64]:
# Make RxNorm table

rx = voc.loc[voc['vocabulary_id']=='RxNorm'].reset_index(drop=True)

rx.shape

(210102, 10)

In [65]:
rx.head()

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,40229458,benzocaine 0.18 MG/MG Oral Gel,Drug,RxNorm,Clinical Drug,S,1042838,20101205,20991231,NaN
1,40229459,benzocaine 0.18 MG/MG Oral Ointment,Drug,RxNorm,Clinical Drug,S,1042847,20101205,20991231,NaN
2,40232055,benzocaine 0.18 MG/MG Oral Ointment [Topicale],Drug,RxNorm,Branded Drug,S,1049224,20110102,20991231,NaN
3,43525657,benzocaine 180 MG/ML Mucous Membrane Topical Solution,Drug,RxNorm,Clinical Drug,S,1371295,20130401,20991231,NaN
4,43525658,benzocaine 180 MG/ML Mucous Membrane Topical Solution [Topicale],Drug,RxNorm,Branded Drug,S,1371298,20130401,20991231,NaN


In [23]:
## CONCEPT_RELATIONSHIP.csv

cr = pd.read_csv(in_path+'CONCEPT_RELATIONSHIP.csv',low_memory=False, delimiter='\t')

cr.shape                 


(13680422, 6)

In [24]:
cr.head()

,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
0,40642538,37395541,Status of,20220128,20991231,NaN
1,40642537,4020566,Status of,20220128,20991231,NaN
2,40642539,4020566,Module of,20220128,20991231,NaN
3,40642538,3561545,Status of,20220128,20991231,NaN
4,40642537,4020567,Status of,20220128,20991231,NaN


In [25]:
cr.invalid_reason.value_counts()

Series([], Name: count, dtype: int64)

In [26]:
cr.relationship_id.value_counts()

relationship_id
Subsumes               1621960
Is a                   1621960
Mapped from            1204962
Maps to                1204962
Status of               636531
                        ...   
Has surface texture          4
Has surface char             4
Surface texture of           4
Before                       1
After                        1
Name: count, Length: 306, dtype: int64

In [28]:
cr.relationship_id.unique()

array(['Status of', 'Module of', 'Energy used by', 'Mapped from',
       'Maps to', 'Subsumes', 'Is a', 'Has system', 'Has property',
       'Has time aspect', 'Has scale type', 'Has component',
       'Has type of service', 'Has kind', 'Has setting', 'Has method',
       'Has subject matter', 'Has Answer', 'Has role',
       'Contained in panel', 'Component of', 'Panel contains',
       'Concept replaces', 'Concept replaced by', 'System of',
       'Answer of', 'Has asso morph', 'Has causative agent',
       'Has pathology', 'Has finding site', 'Has interpretation',
       'Has occurrence', 'Has clinical course', 'Has temporal context',
       'Has relat context', 'Has proc context', 'Has asso proc',
       'Has dir proc site', 'Using finding method', 'Has interprets',
       'Has asso finding', 'Interprets of', 'Finding asso with',
       'Concept same_as to', 'Followed by', 'Has due to',
       'Has indir proc site', 'Has access', 'Has specimen',
       'Has indir morph', 'Using sub

In [50]:
## For the creation of code mapping tables for main concepts, use the following relationship only:
rel_ls = ('Subsumes','Mapped from')

cr = cr.loc[cr['relationship_id'].isin(rel_ls)].reset_index(drop=True)
cr.shape


(2826922, 6)

In [51]:
cr_id1 = cr['concept_id_1'].unique().tolist()
cr_id2 = cr['concept_id_2'].unique().tolist()

cr_id = pd.Series(cr_id1+cr_id2).unique().tolist()
cr_id = np.array(cr_id, dtype=np.int64)
len(cr_id)

1312426

In [52]:
type(cr_id[3])

numpy.int64

In [66]:
rx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210102 entries, 0 to 210101
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   concept_id        210102 non-null  int64 
 1   concept_name      210102 non-null  object
 2   domain_id         210102 non-null  object
 3   vocabulary_id     210102 non-null  object
 4   concept_class_id  210102 non-null  object
 5   standard_concept  190349 non-null  object
 6   concept_code      210102 non-null  object
 7   valid_start_date  210102 non-null  int64 
 8   valid_end_date    210102 non-null  int64 
 9   invalid_reason    0 non-null       object
dtypes: int64(3), object(7)
memory usage: 16.0+ MB


In [67]:
rx.head()

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,40229458,benzocaine 0.18 MG/MG Oral Gel,Drug,RxNorm,Clinical Drug,S,1042838,20101205,20991231,NaN
1,40229459,benzocaine 0.18 MG/MG Oral Ointment,Drug,RxNorm,Clinical Drug,S,1042847,20101205,20991231,NaN
2,40232055,benzocaine 0.18 MG/MG Oral Ointment [Topicale],Drug,RxNorm,Branded Drug,S,1049224,20110102,20991231,NaN
3,43525657,benzocaine 180 MG/ML Mucous Membrane Topical Solution,Drug,RxNorm,Clinical Drug,S,1371295,20130401,20991231,NaN
4,43525658,benzocaine 180 MG/ML Mucous Membrane Topical Solution [Topicale],Drug,RxNorm,Branded Drug,S,1371298,20130401,20991231,NaN


In [68]:
rx_cut = rx.loc[rx['concept_id'].isin(cr_id)].reset_index(drop=True)
rx_cut.shape

(161698, 10)

In [70]:
rx_cut.columns

Index(['concept_id', 'concept_name', 'domain_id', 'vocabulary_id',
       'concept_class_id', 'standard_concept', 'concept_code',
       'valid_start_date', 'valid_end_date', 'invalid_reason'],
      dtype='object')

In [77]:
rx_out = rx_cut[['concept_id', 'concept_name', 'domain_id', 
       'concept_class_id', 'concept_code']].rename(columns={'concept_code':'RxNorm'})

In [78]:
rx_out

,concept_id,concept_name,domain_id,concept_class_id,RxNorm
0,40229458,benzocaine 0.18 MG/MG Oral Gel,Drug,Clinical Drug,1042838
1,40229459,benzocaine 0.18 MG/MG Oral Ointment,Drug,Clinical Drug,1042847
2,40232055,benzocaine 0.18 MG/MG Oral Ointment [Topicale],Drug,Branded Drug,1049224
3,43525657,benzocaine 180 MG/ML Mucous Membrane Topical Solution,Drug,Clinical Drug,1371295
4,43525658,benzocaine 180 MG/ML Mucous Membrane Topical Solution [Topicale],Drug,Branded Drug,1371298
...,...,...,...,...,...
161693,42609793,16 ML docetaxel 10 MG/ML Injection,Drug,Quant Clinical Drug,2679779
161694,42609794,16 ML docetaxel 10 MG/ML Injection [Docivyx],Drug,Quant Branded Drug,2679784
161695,42609795,2 ML docetaxel 10 MG/ML Injection [Docivyx],Drug,Quant Branded Drug,2679786
161696,42609796,8 ML docetaxel 10 MG/ML Injection,Drug,Quant Clinical Drug,2679787


In [79]:
loi_out = loi[['concept_id', 'concept_name', 'domain_id', 
       'concept_class_id', 'concept_code']].rename(columns={'concept_code':'LOINC'})
atc_out = atc[['concept_id', 'concept_name', 'domain_id', 
       'concept_class_id', 'concept_code']].rename(columns={'concept_code':'ATC'})

In [33]:
#snm.to_csv('/Users/feng.jingna/macLocal/Vocabulary/Athena/output/'+'snomed_omop.csv',index=False)
# loi_out.to_csv('/Users/feng.jingna/macLocal/Vocabulary/Athena/output/'+'loinc_omop.csv',index=False)
# atc_out.to_csv('/Users/feng.jingna/macLocal/Vocabulary/Athena/output/'+'atc_omop.csv',index=False)
# rx_out.to_csv('/Users/feng.jingna/macLocal/Vocabulary/Athena/output/'+'rxnorm_omop.csv',index=False)
ekg.to_csv('/Users/feng.jingna/macLocal/Vocabulary/Athena/output/'+'measurement_omop.csv',index=False)


In [34]:
!ls -lh /Users/feng.jingna/macLocal/Vocabulary/Athena/output

total 197520
drwxr-xr-x   8 feng.jingna  staff   256B May  7 15:16 Echo
-rw-r--r--@  1 feng.jingna  staff   384K Apr 15 14:34 atc_omop.csv
-rw-r--r--@  1 feng.jingna  staff    17M Apr 24  2024 athena_condition_concept_small.csv
-rw-r--r--@  1 feng.jingna  staff    21M Apr 15 14:34 loinc_omop.csv
-rw-r--r--   1 feng.jingna  staff    15M May  7 15:43 measurement_omop.csv
drwxr-xr-x  18 feng.jingna  staff   576B May  6 14:19 omop2obo
drwxr-xr-x   3 feng.jingna  staff    96B Sep 19  2024 ontology
-rw-r--r--@  1 feng.jingna  staff    13M Apr 15 14:34 rxnorm_omop.csv
-rw-r--r--@  1 feng.jingna  staff    31M Apr 15 14:15 snomed_omop.csv
